# Init

In [1]:
from utils import *
from SuperGlue import *
p = 11
ks = 50
Use_SIFT = False
# Use the clearest image as the reference image for burst registration
select_ref_img = True
burst_path = '../images/bookshelf'
file_extension = '*.jpg'
gaussian_ksize = 31  # gaussian kernel size
burst = read_burst(burst_path, file_extension)


# Bursr Registration
reference source: `https://learnopencv.com/image-alignment-feature-based-using-opencv-c-python/`

In [2]:
if select_ref_img:
    change_reference(burst)


In [3]:
if Use_SIFT:
    burst = register_burst(burst)
else:
    burst = register_burst_SuperGlue(burst, copy=False, force_cpu=False)


Running inference on device "cuda"
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
process:1/10
process:2/10
process:3/10
process:4/10
process:5/10
process:6/10
process:7/10
process:8/10
process:9/10


In [4]:
# cut the edge in order to remove black pixels
burst = burst[:, 20:-20, 20:-20, :]


In [5]:
# visualize warp result
print(burst.shape)
plt.figure(figsize=(100, 80))
for i in range(burst.shape[0]):
    plt.subplot(math.ceil(burst.shape[0]/3), 3, i+1)
    plt.imshow(cv2.cvtColor(burst[i], cv2. COLOR_BGR2RGB))
plt.savefig('./warp_result.png')


(10, 2161, 3461, 3)


# FBA

In [6]:
# np.moveaxis: change the shape from (# of img, R, C, color) to (# of img, color, R, C)
spectrums = np.fft.fft2(np.moveaxis(burst, 3, 1))
# spectrum.shape = (# of img, color, R, C)

# get the spectrum of a blur kernel
shape = spectrums.shape[-2:]
sig = min(shape)/ks
# blur_kernel_spectrum=get_gau_ker(gaussian_ksize, sig, shape)[1]

# average color channels
weight = np.mean(np.abs(spectrums), axis=1)

# pass through the gaussian filter
weight = np.fft.fftshift(weight)
for i in range(weight.shape[0]):
    weight[i] = cv2.GaussianBlur(weight[i, :, :], (31, 31), sig)
weight = np.fft.ifftshift(weight)

weight = np.power(weight, p)
weight /= np.sum(weight, axis=0)

# expand the shape of the weight from (# of img, R, C) to (# of img, color, R, C)
weight = np.repeat(np.expand_dims(weight, axis=1), 3, axis=1)

# restore image
spectrum_restored = np.sum(weight*spectrums, axis=0)
image_restored = np.fft.ifft2(spectrum_restored)

# change the shape from (color, R, C) to (R, C, color)
image_restored = np.moveaxis(image_restored, 0, 2)

# restore to uint8
image_restored = image_restored.real
print(
    f'# of pixels less than 0: {np.sum(image_restored<0)}\n# of pixels more than 255: {np.sum(image_restored>255)}')
image_restored = np.where(image_restored < 0, 0, image_restored)
image_restored = np.where(image_restored > 255, 255,
                          image_restored).astype(np.uint8)


# of pixels less than 0: 0
# of pixels more than 255: 357794


```
The weight of the first image is the largest in high frequency
```

In [7]:
plt.figure(figsize=(100, 80))
for i in range(burst.shape[0]):
    plt.subplot(math.ceil(burst.shape[0]/3), 3, i+1)
    plt.imshow(np.fft.fftshift(weight[i, 0]), 'magma')
    plt.colorbar()
    plt.xticks(visible=False)
    plt.yticks(visible=False)
plt.savefig('./weight.png')


# Post processing

### attempt to do some post-processing

In [8]:
# do unsharp masking in spatial domain
img_unsharp_masking = unsharp_masking(image_restored, 5/6)
cv2.imwrite('result_post_unsharp_masking.png', img_unsharp_masking)

# do non-local means
img_non_local_means = cv2.fastNlMeansDenoisingColored(
    image_restored, None, 5, 5, 7, 21)
cv2.imwrite('result_post_non_local_denoising.png', img_non_local_means)

# higt-pass filter with different filter
kernel = np.array([[0, -1, 0],
                   [-1, 5, -1],
                   [0, -1, 0]])
img_high_pass = cv2.filter2D(image_restored, -1, kernel)
cv2.imwrite('result_post_high_pass.png', img_high_pass)


True

In [9]:
# input: img_restored
# output: img_restored

# post processing (image sharpening)
# image_sharpen = gaussian_sharpen(image_restored, 500, 0.7)
kernel = np.array([[-1, -1, -1],
                   [-1, 9, -1],
                   [-1, -1, -1]])
image_sharpen = cv2.filter2D(src=image_restored, ddepth=-1, kernel=kernel)


# save image

In [10]:
cv2.imwrite('result.png', image_restored)
cv2.imwrite('result_sharpen.png', image_sharpen)
# plt.figure(figsize=(100,80))
plt.subplot(1, 2, 1)
plt.title('result', fontsize=60)
plt.xticks(visible=False)
plt.yticks(visible=False)
plt.imshow(cv2.cvtColor(image_restored, cv2.COLOR_BGR2RGB))
plt.subplot(1, 2, 2)
plt.xticks(visible=False)
plt.yticks(visible=False)
plt.title('sharpen result', fontsize=60)
plt.imshow(cv2.cvtColor(image_sharpen, cv2.COLOR_BGR2RGB))
plt.savefig('./result_comparison.png')


# Fourier spectrums

In [11]:
imgs = [[cv2.imread(
    os.path.join(burst_path, 'result/out_fba.jpg'), 0), 'Frequency Domain of Paper Results'],
    [cv2.cvtColor(image_restored, cv2.COLOR_BGR2GRAY),
     'Frequency Domain of Our Results'],
    [cv2.cvtColor(image_sharpen, cv2.COLOR_BGR2GRAY), 'Frequency Domain of Our Sharpen Results']]
plt.figure(figsize=(30, 10))
for i in range(len(imgs)):
    plt.subplot(1, 3, 1+i)
    spectrum = np.fft.fftshift(np.fft.fft2(imgs[i][0]))
    plt.imshow(np.log10(np.abs(spectrum)), 'magma')
    plt.colorbar()
    plt.xticks(visible=False)
    plt.yticks(visible=False)
    plt.title(imgs[i][1])
plt.savefig('./freq.png')
